In [8]:
import sys
import os

# This is the fix. It explicitly adds the current directory (your project root)
# to the list of places Python looks for modules.
sys.path.append(os.path.abspath('..'))

import cv2
import numpy as np
import tensorflow as tf
from recommendation_engine.recommender import MusicRecommender

# --- Configuration ---
# These paths are now relative to the app/ folder, which is where the script is.
MODEL_PATH = '../models/cnn_emotion_model.h5'
MUSIC_DATA_PATH = '../data/music_processed/processed_music_tags.csv'

EMOTION_MAP = {
    0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy',
    4: 'sad', 5: 'surprise', 6: 'neutral'
}

class MoodMateApp:
    def __init__(self, model_path, music_data_path):
        self.emotion_model = self._load_emotion_model(model_path)
        self.music_recommender = MusicRecommender(music_data_path)

    def _load_emotion_model(self, path):
        try:
            print("load_emotion_model started successfull")
            model = tf.keras.models.load_model(path)
            print("Emotion model loaded successfully. ✅")
            return model
        except Exception as e:
            print(f"--- 🔴 ERROR loading model ---: {e}")
            return None

    def predict_emotion_from_image(self, image_path):
        if self.emotion_model is None: return None
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None: return None
        img = cv2.resize(img, (48, 48))
        img = img / 255.0
        img = np.expand_dims(img, axis=0)
        img = np.expand_dims(img, axis=-1)
        prediction = self.emotion_model.predict(img)
        return EMOTION_MAP[np.argmax(prediction)]

    def run_full_process(self, test_image_path):
        print("run_full_process started successfull")
        detected_emotion = self.predict_emotion_from_image(test_image_path)
        if detected_emotion:
            print(f"\nDetected Emotion: {detected_emotion.upper()}")
            playlist = self.music_recommender.recommend_songs(detected_emotion)
            if not playlist.empty:
                print("\n--- 🎶 Here is your personalized playlist ---")
                print(playlist[['artist_name', 'title']])
                print("------------------------------------------")

if __name__ == '__main__':
    # Update this path with a real image file to test the system
    TEST_IMAGE_PATH = 'E:/Infosys Springboard Project/Emotion Detection and Music Recommendation System/data/fer2013/test/sad/PrivateTest_366361.jpg' 

    app = MoodMateApp(MODEL_PATH, MUSIC_DATA_PATH)
    app.run_full_process(TEST_IMAGE_PATH)

ModuleNotFoundError: No module named 'recommendation_engine'